In [110]:
import numpy as np
import pandas as pd
import plotly.express as px
import math as mt
import scipy as sp
import streamlit as st
import altair as al

Imported what i know ill need and will add as needed

In [111]:
game_data = pd.read_csv("../games.csv")

In [112]:
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [113]:
game_data.columns = game_data.columns.str.lower()
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Changed all column names to lowercase to make them easier to work with.

In [114]:
name_missing = game_data[game_data['name'].isna()]
name_missing


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Although i will be dropping these as im not 100% sure i am fairly certain after a good amount opf research that entry 659 is 'Mortal Kombat II' as its the only game in 1993 on the GEN that sold 1.78 million copies in us and also had a healthy sale margin in the EU market and entry 14244 is "Battle Mania Daiginjou" as it was the only game on GEN in 1993 to not receive sales in NA or EU as it was not released there and had only 30000 sale sin jp. However again i will be dropping these lines as they are missing far too much information in multiple categories and we cannot retrieve its name i also believe this is why it was left unfilled.

In [115]:
game_data = game_data.dropna(subset=['name'])
game_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


i dropped the previously stated rows 

In [116]:
yor_missing = game_data[game_data['year_of_release'].isna()]
yor_missing.sample(20)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
11000,Demon Chaos,PS2,NaN,Action,0.00,0.00,0.09,0.00,NaN,NaN,NaN
4848,Harvest Moon: Save the Homeland,PS2,NaN,Simulation,0.19,0.15,0.00,0.05,76.0,8.8,E
16329,Brothers in Arms: Furious 4,X360,NaN,Shooter,0.01,0.00,0.00,0.00,NaN,NaN,M
11565,Rock Revolution,Wii,NaN,Misc,0.07,0.00,0.00,0.01,46.0,tbd,T
12209,Build-A-Bear Workshop: Friendship Valley,Wii,NaN,Misc,0.06,0.00,0.00,0.00,NaN,tbd,E
13609,Battle vs. Chess,PS3,NaN,Misc,0.00,0.03,0.00,0.01,NaN,NaN,T
5336,Tomb Raider (2013),PC,NaN,Action,0.06,0.25,0.00,0.04,86.0,8.5,M
12221,Port Royale 3,X360,NaN,Simulation,0.06,0.00,0.00,0.01,56.0,4.5,T
8067,Backbreaker,X360,NaN,Sports,0.17,0.00,0.00,0.01,54.0,7.6,E
14769,Super Duper Sumos,GBA,NaN,Action,0.02,0.01,0.00,0.00,57.0,tbd,E


I Went through a couple fixes to the missing values in the years_of_release column such as using the mean/median, dropping the columns, i learned KNN (K-Nearest Neighbors) but i ultimately decided to leave them their as they skewed the data too much and the information itself makes no sense to be imputed as the release date is a fact and not simply data. I am quite certain this data is missing due to human error as some of these games release years are listed in the title.

In [117]:
critic_missing = game_data[game_data['critic_score'].isna()]
critic_missing.sample(20)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
4522,Monopoly,PS2,2008.0,Misc,0.21,0.16,0.00,0.05,NaN,7.2,E
11948,Itoi Shigesato no Bass Tsuri No. 1 Ketteihan!,N64,2000.0,Sports,0.00,0.00,0.07,0.00,NaN,NaN,NaN
11928,DS Uranai Seikatsu,DS,2009.0,Misc,0.00,0.00,0.07,0.00,NaN,NaN,NaN
4704,Command & Conquer: Red Alert,PS,1997.0,Strategy,0.23,0.16,0.00,0.03,NaN,NaN,NaN
8266,Jewel Match,DS,2010.0,Puzzle,0.05,0.11,0.00,0.02,NaN,tbd,E
15186,Harukanaru Toki no Naka de 3 with Izayoiki Aiz...,PSP,2009.0,Adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN
7390,The Magic School Bus: Oceans,DS,2011.0,Adventure,0.20,0.00,0.00,0.02,NaN,tbd,E
15783,Robotics;Notes,PSV,2014.0,Adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN
11807,And1 Streetball,XB,2006.0,Action,0.06,0.02,0.00,0.00,NaN,NaN,NaN
10594,3D Tank Warfare Simulator Panzer Front,PS,1999.0,Simulation,0.06,0.04,0.00,0.01,NaN,NaN,NaN


In [118]:
game_data['user_score'].value_counts()

user_score
tbd    2424
7.8     324
8       290
8.2     282
8.3     254
       ... 
1.3       2
1         2
0.9       2
0         1
9.7       1
Name: count, Length: 96, dtype: int64

In [119]:
tbd_scores = game_data[game_data['user_score'] == 'tbd']
tbd_scores

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
119,Zumba Fitness,Wii,2010.0,Sports,3.45,2.59,0.0,0.66,NaN,tbd,E
301,Namco Museum: 50th Anniversary,PS2,2005.0,Misc,2.08,1.35,0.0,0.54,61.0,tbd,E10+
520,Zumba Fitness 2,Wii,2011.0,Sports,1.51,1.03,0.0,0.27,NaN,tbd,T
645,uDraw Studio,Wii,2010.0,Misc,1.65,0.57,0.0,0.20,71.0,tbd,E
657,Frogger's Adventures: Temple of the Frog,GBA,NaN,Adventure,2.15,0.18,0.0,0.07,73.0,tbd,E
...,...,...,...,...,...,...,...,...,...,...,...
16695,Planet Monsters,GBA,2001.0,Action,0.01,0.00,0.0,0.00,67.0,tbd,E
16697,Bust-A-Move 3000,GC,2003.0,Puzzle,0.01,0.00,0.0,0.00,53.0,tbd,E
16698,Mega Brain Boost,DS,2008.0,Puzzle,0.01,0.00,0.0,0.00,48.0,tbd,E
16704,Plushees,DS,2008.0,Simulation,0.01,0.00,0.0,0.00,NaN,tbd,E


Because i could need these for future calculations i made a separate dataframe for them in case i need them grouped before i drop the tbd for a missing value

In [120]:
game_data['user_score'] = pd.to_numeric(game_data['user_score'], errors='coerce')


game_data['user_score'].value_counts()


user_score
7.8    324
8.0    290
8.2    282
8.3    254
8.5    253
      ... 
1.3      2
1.0      2
0.9      2
0.0      1
9.7      1
Name: count, Length: 95, dtype: int64

I then coerced them into numeric or 'int' which solved the data type problem of them being an object and the tbd to missing value problem

In [121]:
score_columns = ['critic_score', 'user_score', 'rating']
filtered_scores = game_data[score_columns]
filtered_scores

,critic_score,user_score,rating
0,76.0,8.0,E
1,NaN,NaN,NaN
2,82.0,8.3,E
3,80.0,8.0,E
4,NaN,NaN,NaN
...,...,...,...
16710,NaN,NaN,NaN
16711,NaN,NaN,NaN
16712,NaN,NaN,NaN
16713,NaN,NaN,NaN


In [122]:
filtered_scores_cleaned = filtered_scores.dropna(how='all')

filtered_scores_cleaned


,critic_score,user_score,rating
0,76.0,8.0,E
2,82.0,8.3,E
3,80.0,8.0,E
6,89.0,8.5,E
7,58.0,6.6,E
...,...,...,...
16698,48.0,NaN,E
16702,60.0,7.2,E10+
16704,NaN,NaN,E
16705,63.0,5.8,NaN


I then created a separate dataframe with the critic score user score and rating in one data set because they seemed to be all missing alot of values across the board so i grouped and dropped any row that was missing all 3 and i will use this for the calculations most likely down the line i believe these were left blank as the score do not exist yet indicated by over 200 entries being labeled 'tbd' i infer there is a certain criteria that nominates a game to be reviewed and the closer you are to that criteria the more swiftly this review is given it would explain why the entries listed as 'tbd' have a more robust well of information in the critic and ratings columns as opposed ot the ones filled with simply 'NaN'

Looked for a correlation in missing values before and then i started preparing them as needed